# Clasificación de Planes de Clientes – Megaline  

## Introducción  

La compañía de telecomunicaciones **Megaline** busca reducir el uso de planes heredados y migrar a sus clientes hacia planes más rentables: **Smart** y **Ultra**.  
Para lograrlo, se plantea el desarrollo de un **modelo de clasificación supervisado** capaz de predecir qué plan se ajusta mejor al comportamiento de cada usuario.  

El dataset contiene información sobre llamadas, mensajes y consumo de internet de clientes que ya han migrado a los nuevos planes.  
El objetivo es entrenar, comparar y seleccionar el modelo con mayor exactitud posible, que posteriormente pueda recomendar el plan adecuado para nuevos clientes. 

# Librerias

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. Carga de los datos

In [26]:
# cargamos los datos desde .csv
df= pd.read_csv('users_behavior.csv')

# muestra los 5 primeros registros del dataset
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


# 2. Exploración inicial del dataset

In [27]:
# muestra la informacion general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


El dataset contiene 3,214 registros y 5 columnas. Todas las variables tienen un tipo de dato adecuado: cuatro numéricas de tipo float64 y una categórica binaria de tipo int64.

In [28]:
# comprueba si hay datos faltantes
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

El dataframe no cuenta con valores nulos.

In [29]:
# comprueba si existen datos duplicados
df.duplicated().sum()

0

El dataframe no cuenta con valores duplicados

# 3. Segmentacion de los datos en conjuntos de entrenamiento y validacion
Se utilizara la función train_test_split de la librería scikit-learn para dividir el dataset en dos conjuntos: 70% para entrenamiento y 30% para validación.

In [30]:
# segmentacion de los datos en conjunto de entrenamiento y validacion
f_train, f_valid= train_test_split(df, test_size= 0.3, random_state= 42)

# preparacion de los datos de entrenamiento
X_train= f_train.drop(['is_ultra'], axis= 1) # variables independientes
y_train= f_train['is_ultra'] # variable dependiente

# preparacion de los datos para validacion
X_valid= f_valid.drop(['is_ultra'], axis= 1) # variables independientes
y_valid= f_valid['is_ultra'] # variable dependiente

# muestra el tamaño de los datasets de entrenamiento y prueba
print(f"El dataset para entrenar el modelo cuenta con {X_train.shape[0]} filas y {X_train.shape[1]} columnas.")
print(f"El dataset para probar el modelo cuenta con {X_valid.shape[0]} filas y {X_valid.shape[1]} columnas.")

El dataset para entrenar el modelo cuenta con 2249 filas y 4 columnas.
El dataset para probar el modelo cuenta con 965 filas y 4 columnas.


# 4. Prueba de Modelos
Se probarán distintos modelos de clasificación con el fin de identificar el más adecuado para este proyecto. Se utilizará la validación cruzada mediante GridSearchCV para encontrar la mejor combinación de parámetros.

### Modelo DecisionTreeClassifier

Entrenaminto y seleccion de parametros.

In [31]:
# inicia el modelo
model= DecisionTreeClassifier(random_state= 42)

# parametros para GridSearchCV
param_grid= {
    'max_depth': [3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10, 15],
    'criterion': ['gini', 'entropy']
}

# configuracion de GridSearchCV
grid_search= GridSearchCV(estimator= model, param_grid= param_grid, cv= 5, n_jobs= -1, scoring= 'accuracy')

# entrenar el modelo
grid_search.fit(X_train, y_train)

# mostrar los mejores parametros obtenidos para el modelo DecisionTreeClassifier
print("Mejores parametros", grid_search.best_params_)
print("Mejor resultado obtenido", grid_search.best_score_)

Mejores parametros {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 2}
Mejor resultado obtenido 0.7936807720861172


El modelo DecisionTree optimizado alcanza una exactitud promedio del 79% con los datos de entrenamiento, utilizando criterion='entropy' y max_depth=3. Esto indica un buen balance entre simplicidad y rendimiento.

Evaluacion del modelo en el conjunto de prueba utilizando los parametros optimos.

In [32]:
# inicial el modelo DecisionTreeClassifier con los parametros obtenidos
model= DecisionTreeClassifier(**grid_search.best_params_, random_state= 42)

# entrena el modelo
model.fit(X_train, y_train)

# realiza predicciones con los datos de prueba
predict= model.predict(X_valid)

# evalua el modelo
accuracy= accuracy_score(y_valid, predict)
print(f"La precision del modelo DecisionTreeClassifier es: {accuracy:.2%}")

La precision del modelo DecisionTreeClassifier es: 79.27%


El modelo presenta un rendimiento consistente en los conjuntos de entrenamiento y prueba, lo que indica que no existe sobreajuste significativo.

### Modelo RandomForestClassifier

Entrenaminto y seleccion de parametros.

In [33]:
# inicia el modelo
model= RandomForestClassifier(random_state= 42)

# parametros para GridSearchCV
param_grid= {
    'n_estimators': [25, 50, 100, 150, 200],
    'max_depth': [3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10, 15],
    'criterion': ['gini', 'entropy']
}

# configuracion de GridSearchCV
grid_search= GridSearchCV(estimator= model, param_grid= param_grid, cv= 5, n_jobs= -1, scoring= 'accuracy')

# entrenar el modelo
grid_search.fit(X_train, y_train)

# mostrar los mejores parametros obtenidos para el modelo RandomForestClassifier
print("Mejores parametros", grid_search.best_params_)
print("Mejor resultado obtenido", grid_search.best_score_)



Mejores parametros {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 150}
Mejor resultado obtenido 0.8123494184607771


El modelo alcanza una exactitud del 81%, lo que significa que clasifica correctamente 8 de cada 10 clientes en promedio, mostrando un desempeño sólido y generalizable para la tarea.

Evaluacion del modelo en el conjunto de prueba utilizando los parametros optimos.

In [34]:
# inicia el modelo RandomForestClassifier con los parametros obtenidos
model= RandomForestClassifier(**grid_search.best_params_, random_state= 42)

# entrena el modelo
model.fit(X_train, y_train)

# realiza predicciones con los datos de prueba
predict= model.predict(X_valid)

# evalua el modelo
accuracy= accuracy_score(y_valid, predict)
print(f"La precision del modelo RandomForestClassifier es: {accuracy:.2%}")

La precision del modelo RandomForestClassifier es: 80.73%


El modelo muestra un rendimiento similar en los conjuntos de entrenamiento y prueba, lo que indica una buena capacidad de generalización y ausencia de sobreajuste significativo.

### Modelo LogisticRegression

Entrenaminto y seleccion de parametros.

In [35]:
# inicia el modelo
model= LogisticRegression(random_state= 42)

# parametros para GridSearchCV
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# configuracion de GridSearchCV
grid_search= GridSearchCV(estimator= model, param_grid= param_grid_lr, cv= 5, n_jobs= -1, scoring= 'accuracy')

# entrena el modelo
grid_search.fit(X_train, y_train)

# mostrar los mejores parametros obtenidos para el modelo LogisticRegression
print("Mejores parametros", grid_search.best_params_)
print("Mejor resultado obtenido", grid_search.best_score_)

Mejores parametros {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Mejor resultado obtenido 0.7461123484286067


El modelo de regresión logística optimizado con C=1, penalty='l1' y solver='liblinear' alcanzó una exactitud promedio del 74.6% en validación cruzada. Esto refleja un desempeño competitivo, mostrando que la regularización L1 contribuye a simplificar el modelo y mejorar su capacidad de generalización.

Evaluacion del modelo en el conjunto de prueba utilizando los parametros optimos.

In [36]:
# inicia el modelo LogisticRegression con los parametros obtenidos
model= LogisticRegression(**grid_search.best_params_, random_state= 42)

# entrena el modelo
model.fit(X_train, y_train)

# realiza predicciones con los datos de prueba
predict= model.predict(X_valid)

# evalua el modelo
accuracy= accuracy_score(y_valid, predict)
print(f"La precision del modelo LogisticRegression es: {accuracy:.2%}")

La precision del modelo LogisticRegression es: 74.40%


El modelo de regresión logística optimizado logra un rendimiento estable y consistente entre validación y prueba (~74%). Esto muestra que es una opción sólida para clasificar clientes en planes de Megaline, ofreciendo un equilibrio entre simplicidad, interpretabilidad y capacidad predictiva."

# Conclusion
Se evaluaron tres modelos de clasificación (Logistic Regression, DecisionTreeClassifier y RandomForestClassifier). Tras el ajuste de hiperparámetros con GridSearchCV, el RandomForestClassifier con criterion='gini', max_depth=10, min_samples_split=10 y n_estimators=150 mostró el mejor desempeño, alcanzando un 81.2% de exactitud en validación y 80.7% en los datos de prueba. Por lo tanto, este modelo se considera la mejor opción para la tarea de clasificación.